# 1. Importing Dependencies and Setting Up Environment

In [35]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import math
import pandas as pd
import regex as re
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm

In [36]:
torch.__version__

'2.5.1+cu124'

In [37]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [38]:
torch.cuda.is_available()

True

# 2. Reading and Processing Dataset

In [39]:
df = pd.read_parquet('/kaggle/input/transliteration-en-bn/transliteration.parquet')
df # bn is bengali font, rm is 

,bn,rm
0,স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???,scroll kore 20/30 second er video pann nai???
1,ও গুলা টরেন্ট সাইট এ পাবেন,o gula Torrent site e paben
2,ভক্কর চক্কর পোস্ট একটা করলেই এপ্রুভড.… নিশ্চই ...,vokkor chokkor post akta korlei approved…. nis...
3,আমি টেস্ট করেই কোড দিছি…,ami test koreii code disi…
4,"এতো কষ্টের কি আছে সাকিবওয়াপ.টক,সাকিবওয়াপ.মল&এআ...","eto koster ki ache shakibwap.tk,shakibwap.ml&a..."
...,...,...
5001,এদের কে মাটি চাপা দেয়া দরকার,Eder ke mati chapa deya dorkar
5002,ওদের একদিন বিচার হবে ভাই,Oder ekdin bichar hobe vai
5003,অপেক্ষা করেন,Opekkha koren
5004,সময় আসবে,Shomoy ashbe


In [40]:
df.rm = df.rm.str.lower()

In [41]:
bn_nums = [chr(i) for i in range(2534, 2544)]
rm_nums = [str(i) for i in range(10)]
regex = rf"""(\W)|({'|'.join(bn_nums)})|({'|'.join(rm_nums)})"""
# regex = rf"""."""

def split_to_tokens(text):
    # splits = re.split(regex, text)
    splits = list(text)
    return [t for t in splits]

def build_vocab(data_list):
    words = []
    for data in data_list:
        splits = split_to_tokens(data)
        words.extend(splits)
    vocab = list(map(lambda x: x.lower() , list(set(words))))
    return vocab

In [42]:
bn_vocab = build_vocab(df.bn.tolist())
rm_vocab = build_vocab(df.rm.tolist())
len(bn_vocab), len(rm_vocab)

(108, 97)

In [43]:
# register special characters at 0, 1, 2, 3
specials = ['<PAD>', '<UNK>', '<BOS>', '<EOS>']
ln_special = len(specials)
bn_lookup = {}
rm_lookup = {}
bn_idx2chr = {}
rm_idx2chr = {}

for i,c in enumerate(specials):
    bn_lookup[c]=i
    rm_lookup[c]=i
    bn_idx2chr[i]=c
    rm_idx2chr[i]=c

for i,c in enumerate(bn_vocab):
    bn_lookup[c] = i+ln_special
    bn_idx2chr[i+ln_special]=c

for i,c in enumerate(rm_vocab):
    rm_lookup[c] = i+len(specials)
    rm_idx2chr[i+ln_special]=c


In [44]:
def tokens2idx(tokens : list, lookup2idx : dict)->list:
    return [lookup2idx[t] for t in tokens]

def idx2tokens(idxs : list, idx2tokens : dict):
    return [idx2tokens[int(t)] for t in idxs]

def prepare_data(texts, lookup2idx):
    data = []
    for text in texts:
        splits = split_to_tokens(text)
        data.append(tokens2idx(splits, lookup2idx))
    return data

In [45]:
bn_data = prepare_data(df.bn.tolist(), bn_lookup)
rm_data = prepare_data(df.rm.tolist(), rm_lookup)
data = list(zip(bn_data, rm_data))

In [46]:
max(len(t) for t in bn_data),max(len(t) for t in rm_data)

(246, 253)

In [47]:
for bn, rm in data:
    print(idx2tokens(bn, bn_idx2chr))
    print(idx2tokens(rm, rm_idx2chr))
    break

['স', '্', 'ক', '্', 'র', 'ো', 'ল', ' ', 'ক', 'র', 'ে', ' ', '২', '০', '/', '৩', '০', ' ', 'স', 'ে', 'ক', 'ে', 'ন', '্', 'ড', ' ', 'এ', 'র', ' ', 'ভ', 'ি', 'ড', 'ি', 'ও', ' ', 'প', 'া', 'ন', ' ', 'ন', 'া', 'ই', '?', '?', '?']
['s', 'c', 'r', 'o', 'l', 'l', ' ', 'k', 'o', 'r', 'e', ' ', '2', '0', '/', '3', '0', ' ', 's', 'e', 'c', 'o', 'n', 'd', ' ', 'e', 'r', ' ', 'v', 'i', 'd', 'e', 'o', ' ', 'p', 'a', 'n', 'n', ' ', 'n', 'a', 'i', '?', '?', '?']


In [48]:
len(list(df.bn[0]))

45

In [49]:
# hyper params
max_seq_len = 256 # considering the decoder input and output would be sliced from it by one step
batch_size = 128
train_size = 0.8
d_model = 128

In [50]:
train_data, test_data = train_test_split(data, train_size=train_size, random_state=42)

In [51]:
def add_BOS_EOS(tokens : torch.LongTensor):
    return torch.tensor([torch.tensor(bn_lookup['<BOS>']), *tokens, torch.tensor(bn_lookup['<EOS>'])])

def pad_or_trunc(tokens : torch.LongTensor, max_seq_len : int):
    len_ = tokens.shape[0]
    if len_>max_seq_len:
        return tokens[:max_seq_len]
    else:
        pad_len = max_seq_len-len_
        return torch.cat([tokens, torch.zeros(pad_len)], dim=0)

def pad_and_generate_mask(batch):
    bn_batch, rm_batch, bn_mask_batch, rm_mask_batch = [], [], [], []
    for bn_elem, rm_elem in batch:
        bn_elem = add_BOS_EOS(bn_elem)
        rm_elem = add_BOS_EOS(rm_elem)

        bn_elem = pad_or_trunc(bn_elem, max_seq_len=max_seq_len+1).type(torch.long)
        rm_elem = pad_or_trunc(rm_elem, max_seq_len=max_seq_len).type(torch.long)

        bn_mask = (bn_elem!=bn_lookup['<PAD>']).type(torch.long) # anything not padding will be 1
        rm_mask = (rm_elem!=rm_lookup['<PAD>']).type(torch.long) # anything not padding will be 1

        bn_batch.append(bn_elem)
        rm_batch.append(rm_elem)
        bn_mask_batch.append(bn_mask)
        rm_mask_batch.append(rm_mask)
    
    bn_batch = torch.stack(bn_batch, dim=0)
    rm_batch = torch.stack(rm_batch, dim=0)

    bn_mask_batch = torch.stack(bn_mask_batch, dim=0)
    rm_mask_batch = torch.stack(rm_mask_batch, dim=0)

    return rm_batch, bn_batch, rm_mask_batch, bn_mask_batch


In [52]:
train_data_loader = DataLoader(train_data, batch_size=batch_size, collate_fn=pad_and_generate_mask)
test_data_loader = DataLoader(test_data, batch_size=batch_size, collate_fn=pad_and_generate_mask)

In [53]:
for rm_batch, bn_batch, rm_mask_padding, bn_mask_batch in train_data_loader:
    break
# rm_batch = rm_batch.to(device)
# rm_mask_padding = rm_mask_padding.to(device)
# bn_batch = bn_batch.to(device)
# bn_mask_batch = bn_mask_batch.to(device)
rm_batch.shape, rm_mask_padding.shape, bn_batch.shape, bn_mask_batch.shape

(torch.Size([128, 256]),
 torch.Size([128, 256]),
 torch.Size([128, 257]),
 torch.Size([128, 257]))

# 3. Core Components of Transformer

In [54]:
class RMEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(RMEmbedding, self).__init__()
        self.embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
    
    def forward(self, x):
        return self.embedding_table(x)
    
class BenEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(BenEmbedding, self).__init__()
        self.embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
    
    def forward(self, x):
        return self.embedding_table(x)
    
class PositionalEmbedding(nn.Module):
    def __init__(self, max_seq_len, embedding_dim):
        super(PositionalEmbedding, self).__init__()
        self.max_seq_len = max_seq_len
        self.embedding_dim = embedding_dim

        self.PE = self.get_pos_emb()

        self.register_buffer('pe', self.PE)

    def get_pos_emb(self):
        T = self.max_seq_len
        D = self.embedding_dim

        PE = torch.zeros(1, T, D) # generating for one example and will be broadcasted for batch dimension
        pos = torch.arange(0, T, 1).view(T, 1)
        dim = torch.arange(0, D, 1).view(1, D)

        division_term = 100**(2*dim/D)

        PE[:, 0::2] = torch.sin(pos[0::2]/division_term)
        PE[:, 1::2] = torch.cos(pos[1::2]/division_term)

        return PE.to(device)


    def forward(self, x):
        seq_len = x.shape[1]
        return x + self.PE[:, :seq_len, :] # add till the seq length available

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, dff):
        super(PositionWiseFeedForward, self).__init__()
        self.ff = nn.Sequential(
            nn.Linear(in_features=d_model, out_features=dff),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=dff, out_features=d_model)
        )
    
    def forward(self, x):
        return self.ff(x)

class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k):
        super(ScaledDotProductAttention, self).__init__()
        self.d_k = d_k
    
    def forward(self, Q, K, V, mask=None):
        # used -2, -1 for transpose as this can be used for both multihead splits or without splits
        scores = torch.matmul(Q, K.transpose(-2, -1))/math.sqrt(self.d_k) 
        if mask is not None:
            scores = scores.masked_fill(mask=(mask==0), value=-1e9)
        
        attention_probs = F.softmax(scores, dim=-1)
        output = torch.matmul(attention_probs, V)

        return output

class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, d_model, d_k):
        super(MultiHeadAttention, self).__init__()
        self.n_heads = n_heads
        self.d_model = d_model
        self.d_k = d_k
        self.d_v = d_model//n_heads

        # no bias as only weights are multiplied to the QKV and Output metrix
        self.W_q = nn.Linear(in_features=d_model, out_features=d_k*n_heads, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_k*n_heads, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=self.d_v*n_heads, bias=False)
        self.W_o = nn.Linear(in_features=self.d_v*n_heads, out_features=d_model, bias=False)

        self.attn = ScaledDotProductAttention(self.d_k)
    
    def split_heads(self, x):
        """Expects input in shape of B,T,d_k*heads"""
        batch, seq_len, dim = x.shape
        # first view as to break the features in to n_heads then separate the heads as transpose
        x = x.view(batch, seq_len, self.n_heads, self.d_k).transpose(1, 2)
        return x
    def combine_heads(self, x):
        """Expects input in shape of B,T,heads,d_v"""
        B, n_heads, T, d_v = x.shape
        return x.transpose(1,2).contiguous().view(B, T, d_v*self.n_heads) # getting the shape of B, T, d_model
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attented_output = self.attn(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attented_output))
        return output

class AddAndLayerNorm(nn.Module):
    def __init__(self, d_model):
        super(AddAndLayerNorm, self).__init__()
        self.d_model = d_model
        self.lmn = nn.LayerNorm(d_model)

    def forward(self, x, sublayer_x):
        return self.lmn(x+sublayer_x)

In [55]:
class EncoderBlock(nn.Module):
    def __init__(self, d_model, d_k, n_heads, dropout):
        super(EncoderBlock, self).__init__()
        self.d_model = d_model
        self.d_k = d_k
        self.n_heads = n_heads
        self.dropout = dropout

        self.self_attn = MultiHeadAttention(n_heads=n_heads, d_model=d_model, d_k=d_k)
        self.add_n_norm1 = AddAndLayerNorm(d_model=d_model)
        self.add_n_norm2 = AddAndLayerNorm(d_model=d_model)
        self.ff = PositionWiseFeedForward(d_model=d_model, dff=d_model*4) # as per section 3.3 of base paper
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask):
        attented = self.self_attn(x, x, x, mask)
        norm1 = self.add_n_norm1(x, self.dropout1(attented))

        ff = self.ff(norm1)
        norm2 = self.add_n_norm2(norm1, self.dropout2(ff))

        return norm2

In [56]:
class DecoderBlock(nn.Module):
    def __init__(self, d_model, d_k, n_heads, dropout):
        super(DecoderBlock, self).__init__()
        self.d_model = d_model
        self.d_k = d_k
        self.n_heads = n_heads
        self.dropout = dropout

        self.self_attn = MultiHeadAttention(n_heads=n_heads, d_model=d_model, d_k=d_k)
        self.add_n_norm1 = AddAndLayerNorm(d_model=d_model)
        self.cross_attn = MultiHeadAttention(n_heads=n_heads, d_model=d_model, d_k=d_k)
        self.add_n_norm2 = AddAndLayerNorm(d_model=d_model)
        self.ff = PositionWiseFeedForward(d_model=d_model, dff=d_model*4)
        self.add_n_norm3 = AddAndLayerNorm(d_model=d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, enc_output, src_mask, tgt_mask):
        self_attented = self.self_attn(x, x, x, tgt_mask)
        x = self.add_n_norm1(x, self.dropout(self_attented))

        cross_attented = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.add_n_norm2(x, self.dropout(cross_attented))

        projected = self.ff(x)
        x = self.add_n_norm3(x, self.dropout(projected))

        return x

# 4. Building Transformer

In [57]:
class Transformer(nn.Module):
    def __init__(self, rm_vocab_size, bn_vocab_size, d_model, max_seq_len, n_enc_layer=6, n_dec_layer=6, d_k=512, n_heads=8, dropout=0.5):
        super(Transformer, self).__init__()
        self.rm_embedding = RMEmbedding(rm_vocab_size, d_model)
        self.bn_embedding = BenEmbedding(bn_vocab_size, d_model)
        self.positional_embedding = PositionalEmbedding(max_seq_len=max_seq_len, embedding_dim=d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        self.encoder_layers = nn.ModuleList([EncoderBlock(d_model=d_model, d_k=d_k, n_heads=n_heads, dropout=dropout) for _ in range(n_enc_layer)])
        self.decoder_layers = nn.ModuleList([DecoderBlock(d_model=d_model, d_k=d_k, n_heads=n_heads, dropout=dropout) for _ in range(n_dec_layer)])

        self.final_project = nn.Sequential(
            nn.Linear(in_features=d_model, out_features=bn_vocab_size),
            # nn.Softmax(dim=-1) # as we are using CrossEntropyLoss which considers logsoftmax by default
        )
    
    def gen_attn_mask(self, rm_mask_batch, bn_mask_batch):
        B,T = bn_mask_batch.shape

        rm_mask_batch = rm_mask_batch.unsqueeze(1)
        bn_mask_batch = bn_mask_batch.unsqueeze(1)

        no_peak_future_mask = torch.tril(torch.ones(B, T, T), diagonal=0).type(torch.long).to(device)

        # .unsqueeze(1) at the end of the lines would add n_heads dimension in the masks as 1 and broadcasted later
        bn_mask = (bn_mask_batch & no_peak_future_mask).unsqueeze(1)

        rm_mask = (rm_mask_batch.unsqueeze(-1)&rm_mask_batch.unsqueeze(-2)) # create dimension in 1 and 2 separately and do a broadcasting and operation
        return rm_mask.to(device), bn_mask.to(device)
    
    def forward(self, rm_batch, bn_batch, rm_mask_batch, bn_mask_batch):
        rm_mask_padding, bn_mask_causal = self.gen_attn_mask(rm_mask_batch=rm_mask_batch, bn_mask_batch=bn_mask_batch)
        rm_embedding = self.dropout1(self.positional_embedding(self.rm_embedding(rm_batch)))
        bn_embedding = self.dropout2(self.positional_embedding(self.bn_embedding(bn_batch)))

        enc_output = rm_embedding
        for enc_block in self.encoder_layers:
            enc_output = enc_block(enc_output, mask=rm_mask_padding)
        
        dec_output = bn_embedding
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask=rm_mask_padding, tgt_mask = bn_mask_causal)
        
        output = self.final_project(dec_output)

        return output
    
    def transliterate(self, rm_text, max_output_len=max_seq_len, stop_idx = 3):
        """Can be used for one sentence"""
        self.eval()
        with torch.no_grad():
            rm_tokens = [[rm_lookup['<BOS>'], *tokens] for tokens in prepare_data([rm_text.lower()], rm_lookup)]
            
            tokens = torch.tensor(rm_tokens, dtype=torch.long, device=device)
            rm_mask = None # one sentence, no padding mask needed to ignore

            enc_out = self.positional_embedding(self.rm_embedding(tokens))
            for enc_block in self.encoder_layers:
                enc_out = enc_block(enc_out, mask=rm_mask)
            
            bn_tokens = [bn_lookup['<BOS>']]
            for i in range(max_output_len):
                # print(bn_tokens)
                bn_token_tensor = torch.tensor([bn_tokens[-256:]], dtype=torch.long, device=device) # convert to 2D tensor
                bn_out = self.positional_embedding(self.bn_embedding(bn_token_tensor))
                
                B, T = bn_token_tensor.shape
                bn_causal_mask = torch.tril(torch.ones(B, T, T)).type(torch.long).to(device)

                for dec_block in self.decoder_layers:
                    bn_out = dec_block(bn_out, enc_out, src_mask = None, tgt_mask = bn_causal_mask)
                output = self.final_project(bn_out[:, -1:, :]) # only project the last generated token
                idxs = output.argmax(dim=-1) # B, T
                last_predicted_idx = idxs[0,-1].item()
                bn_tokens.append(last_predicted_idx)
                if last_predicted_idx==bn_lookup['<EOS>']:
                    break
        
        text_gen = idx2tokens(bn_tokens, bn_idx2chr)
        return "".join(text_gen)


# 5. Defining Hyper Parameters and Model Instance

In [79]:
rm_vocab_size = len(rm_lookup)
bn_vocab_size = len(bn_lookup)
num_heads=8
d_k = d_model//num_heads
dropout = 0.3
lr = 0.001 # for now ignoring the LR Scheduler
betas = (0.9, 0.98)
eps = 1e-9
epochs = 20

In [59]:
transformer = Transformer(rm_vocab_size=rm_vocab_size, bn_vocab_size=bn_vocab_size, d_model=d_model, 
                          max_seq_len=max_seq_len, n_enc_layer=6, n_dec_layer=6, d_k=d_k, n_heads=num_heads, dropout=dropout).to(device)

In [60]:
transformer

Transformer(
  (rm_embedding): RMEmbedding(
    (embedding_table): Embedding(101, 128)
  )
  (bn_embedding): BenEmbedding(
    (embedding_table): Embedding(112, 128)
  )
  (positional_embedding): PositionalEmbedding()
  (dropout1): Dropout(p=0.3, inplace=False)
  (dropout2): Dropout(p=0.3, inplace=False)
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderBlock(
      (self_attn): MultiHeadAttention(
        (W_q): Linear(in_features=128, out_features=128, bias=False)
        (W_k): Linear(in_features=128, out_features=128, bias=False)
        (W_v): Linear(in_features=128, out_features=128, bias=False)
        (W_o): Linear(in_features=128, out_features=128, bias=False)
        (attn): ScaledDotProductAttention()
      )
      (add_n_norm1): AddAndLayerNorm(
        (lmn): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (add_n_norm2): AddAndLayerNorm(
        (lmn): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (ff): PositionWiseFeedForward(

In [61]:
# rm_batch.device

In [62]:
# y = transformer(rm_batch, bn_batch[:, :256], rm_mask_padding, bn_mask_batch[:, :256])

In [63]:
# y.shape

In [64]:
transformer.transliterate('Ami tomake valobasi na!!', max_output_len=256, stop_idx=3)

'<BOS>৭থ<PAD>ধ?রঐজ।–গততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততততত'

# 6. Loss Function and Optimizer Definition

In [65]:
optimizer = torch.optim.Adam(transformer.parameters(), lr=lr, betas=betas, eps=eps)
loss_fn = nn.CrossEntropyLoss(ignore_index=0) # ignore padding index for calculating the loss

In [66]:
# bn_batch.type()

In [67]:
# loss_fn(y.view(-1, bn_vocab_size), (bn_batch[:, 1:]).flatten().type(torch.long))

In [68]:
# (bn_batch[:, 1:]).flatten()

In [69]:
def accuracy(y_true, y_pred):
    """use argmax to the logits from model before passing it.
    y_true : (B, T), y_pred : (B, T)"""
    equality = (y_true==y_pred).float()
    mean = torch.mean(equality, dim=1)
    mean = mean.mean(dim=0)
    return mean

In [78]:
texts = [
    "Tumi amake rokto dao, ami tomake swadhinota debo.",
    "Makhon kumar onke 100 te 98 peyeche.",
    "Ami tomar bhalobasha chai, kichu beshi na.",
    "Ei poth cholte cholte raat hoye gelo.",
    "Shobuj paharer gobhire ekta golpo lukiye ache.",
    "Kalke amar porikkha, kintu ami ekhono pori nai.",
    "Ma bollo barite deri kore ashte hobe na."
]

# 7. Training Loop and Training Time Example Evaluation

In [80]:
for i in range(epochs):
    print(f"Epoch: {i+1}/{epochs}")
    mean_loss = 0
    mean_accuracy = 0
    total_b = 0
    transformer.train()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()  # Releases inter-process memory (for multiprocessing)
    for rm_batch, bn_batch, rm_mask_padding, bn_mask_batch in tqdm(train_data_loader):
        rm_batch=rm_batch.to(device, non_blocking=True)
        bn_batch=bn_batch.to(device, non_blocking=True)
        rm_mask_padding=rm_mask_padding.to(device, non_blocking=True)
        bn_mask_batch=bn_mask_batch.to(device, non_blocking=True)

        # print(total_b)
        B = rm_batch.shape[0]
        optimizer.zero_grad()

        bn_batch_in = bn_batch[:, :-1] # except the last step
        bn_mask_batch_in = bn_mask_batch[:, :-1] # except the last step
        bn_batch_out = bn_batch[:, 1:] # everything excep the first tokens
        # bn_mask_batch_out = bn_mask_batch[:, 1:] # everything except the first step
        bn_batch_out_pred = transformer(rm_batch, bn_batch_in, rm_mask_padding, bn_mask_batch_in)

        loss_ = loss_fn(bn_batch_out_pred.view(-1, bn_vocab_size), bn_batch_out.flatten().type(torch.long))
        accuracy_ = accuracy(bn_batch_out_pred.argmax(dim=-1), bn_batch_out).item()
        
        mean_loss += loss_.item()*B
        mean_accuracy += accuracy_*B

        loss_.backward()
        optimizer.step()

        total_b += B

    print(f"TRAIN: Mean loss: {mean_loss/total_b}, Mean Accuracy: {mean_accuracy/total_b}")
    mean_loss = 0
    mean_accuracy = 0
    total_b = 0
    transformer.eval()
    with torch.no_grad():
        for rm_batch, bn_batch, rm_mask_padding, bn_mask_batch in tqdm(test_data_loader):
            rm_batch=rm_batch.to(device, non_blocking=True)
            bn_batch=bn_batch.to(device, non_blocking=True)
            rm_mask_padding=rm_mask_padding.to(device, non_blocking=True)
            bn_mask_batch=bn_mask_batch.to(device, non_blocking=True)

            # print(total_b)
            B = rm_batch.shape[0]

            bn_batch_in = bn_batch[:, :-1] # except the last step
            bn_mask_batch_in = bn_mask_batch[:, :-1] # except the last step
            bn_batch_out = bn_batch[:, 1:] # everything excep the first tokens
            # bn_mask_batch_out = bn_mask_batch[:, 1:] # everything except the first step
            bn_batch_out_pred = transformer(rm_batch, bn_batch_in, rm_mask_padding, bn_mask_batch_in)

            loss_ = loss_fn(bn_batch_out_pred.view(-1, bn_vocab_size), bn_batch_out.flatten().type(torch.long))
            accuracy_ = accuracy(bn_batch_out_pred.argmax(dim=-1), bn_batch_out).item()
            
            mean_loss += loss_.item()*B
            mean_accuracy += accuracy_*B

            total_b += B
    print(f"TEST: Mean loss: {mean_loss/total_b}, Mean Accuracy: {mean_accuracy/total_b}")
    for text in texts:
        print(transformer.transliterate(text, max_output_len=128, stop_idx=3))
    # print(transformer.transliterate('Tumi ki bokachoda?', max_output_len=256, stop_idx=3))

Epoch: 1/20


100%|██████████| 32/32 [00:19<00:00,  1.63it/s]


TRAIN: Mean loss: 0.1429974644851464, Mean Accuracy: 0.13554999691533756


100%|██████████| 8/8 [00:02<00:00,  3.61it/s]


TEST: Mean loss: 0.5132602936850337, Mean Accuracy: 0.12622021662141986
<BOS>তুমি আমাকে রুট দেও, আমি তোকমাধা সাইনা দেবো। <EOS>
<BOS>মাখন কমার অনেক ১০০ টি৩ ৯৮ পেয়েছে।<EOS>
<BOS>আমি তোমার ভালোবাসা চাই, কিছু বেশি না। <EOS>
<BOS>এই পোট চলতে চলতে রাহাতে গেলে গেলো।<EOS>
<BOS>সবুঝাও পারের গবীর একটা গুল্প লিখে আছে।<EOS>
<BOS>কালকে আমার পরিখা, কিন্তু আমি এখনো পরি নাই।<EOS>
<BOS>মা বললি বাড়িট দেরি করে আসতে হবে না।<EOS>
Epoch: 2/20


...

TEST: Mean loss: 0.5066667565566575, Mean Accuracy: 0.12681667914171657
<BOS>টিউমি আমাকে রুট দেও, আমি তোকামা সুধা নিতা দেবো।<EOS>
<BOS>মাখন কমার অনেক ১০০ টে ৯জিপি পেয়েছে।<EOS>
<BOS>আমি তোমার ভালোবাসা চাই, কিছু বেশি না। <EOS>
<BOS>এই পোট চলতে চলতে রাহ হয়ে গেলো। <EOS>
<BOS>সবুঝে পারের গবহইরে একটা গত গুল্প লিখে আছে।<EOS>
<BOS>কালকে আমার পরিক্ষা, কিন্তু আমি এখনো পরি নাই।<EOS>
<BOS>মা বললো বারিট দেরি করে আসতে হবে না। <EOS>
Epoch: 20/20


100%|██████████| 32/32 [00:19<00:00,  1.63it/s]


TRAIN: Mean loss: 0.12005669479216491, Mean Accuracy: 0.13657533873121935


100%|██████████| 8/8 [00:02<00:00,  3.60it/s]


TEST: Mean loss: 0.53924678471274, Mean Accuracy: 0.1257913854724157
<BOS>টামি আমাকে রুট দেও, আমি তোকাম সাধা নিত দে তবো।<EOS>
<BOS>মাখন কমার অনেক ১০০০ টে ৯৮ পেয়েছে।<EOS>
<BOS>আমি তোমার ভালোবাসা চাই, কিছু বেশি না। <EOS>
<BOS>এই পোথ চলতে চলতে রাহ হয়ে গেলো। গেলো।<EOS>
<BOS>সবুঝুজ পাহারের গবেরে একটা গুলো লাকিয়ে আছে।<EOS>
<BOS>কালকে আমার পরিকা, কিন্তু আমি এখনো পরি নাই।<EOS>
<BOS>মা বলললে বারিট দেরি করে আসতে হবে না।<EOS>


# 8. Test Time Example Tests

In [90]:
texts = [
    # Swami Vivekananda
    "Utho, jago, ebong lokkho arjon na kora porjonto themo na.",
    "Shokti holo jibon, durbolota holo mrityu.",
    "Tumi ja bhabo, tai tumi hoye jao.",
    "Ekti dharona grohon koro, seta niye shopno dekho, seta niye bacho.",
    "Jodi ekta din kono samoshyar mukhomukhi na hao, tahole bujhte hobe tumi bhul pothe cholcho.",
    "Shob shokti tomari moddhe ache, etar upor bishwas rakho.",
    "Jodi tumi eksho bar byartho hao, tobu chesta chaliye jao.",
    "Jibon holo songram, ebong songram chara kono agragoti nei.",
    "Tumi ja bhabo, tai tumi hoye jao.",
    "Shokti holo jibon, durbolota holo mrityu.",

    # Rabindranath Tagore
    "Jodi tumi surjo hariye kande, tahole chokher jol tomake tara dekhte dibe na.",
    "Jodi tor daak shune keu na ashe, tobe ekla cholo re.",
    "Tumi jodi shob bhul er dorja bondho koro, tahole shotti o probesh korte parbe na.",
    "Tumi shudhu jol er dike takiye thakle, samudra par korte parbe na.",
    "Manush ekta shishu, tar shokti holo briddhi.",
    "Bristi poreche, ami chatri niye beriyechi!",
    "Ei poth cholte cholte raat hoye gelo.",
    "Tumi amake rokto dao, ami tomake swadhinota debo.",
    "Ami tomar bhalobasha chai, kichu beshi na.",
    "Kalke amar porikkha, kintu ami ekhono pori nai."
]
for text in texts:
    print('===================================================================================================================================')
    print("BANGLISH:=>>", text)
    print("BANGLA:=>>",transformer.transliterate(text, max_output_len=256, stop_idx=3))
    print('===================================================================================================================================')

BANGLISH:=>> Utho, jago, ebong lokkho arjon na kora porjonto themo na.
BANGLA:=>> <BOS>উঠ, যাগো, এবং লক আরো এর জন্য করা পর্যন্ত থেকে তামন না।<EOS>
BANGLISH:=>> Shokti holo jibon, durbolota holo mrityu.
BANGLA:=>> <BOS>সক্ষতি হলো জীবন, ডার্বলতা হলো মরিটিউব <EOS>
BANGLISH:=>> Tumi ja bhabo, tai tumi hoye jao.
BANGLA:=>> <BOS>টুমি যা ভাবো, তাই তোমি হয়ে যাও যাও।<EOS>
BANGLISH:=>> Ekti dharona grohon koro, seta niye shopno dekho, seta niye bacho.
BANGLA:=>> <BOS>একটি ধারনা গ্রহন করো, সেটা নিয়ে সহপন্য দেখো সে টা নিয়ে ভাই হবে।<EOS>
BANGLISH:=>> Jodi ekta din kono samoshyar mukhomukhi na hao, tahole bujhte hobe tumi bhul pothe cholcho.
BANGLA:=>> <BOS>যদি একটা দিন কোনো সাময়ার মুষয় কুমি না হা হা, তাহলে বুঝলে হতে হবে বুঝলে মো টা হল্প হতো। চালে চলে ছি। <EOS>
BANGLISH:=>> Shob shokti tomari moddhe ache, etar upor bishwas rakho.
BANGLA:=>> <BOS>সব ষ্ক্কি তোমারি মধ্যে আছে, এটার উপর বিশ হওয়ার কার কাছে। <EOS>
BANGLISH:=>> Jodi tumi eksho bar byartho hao, tobu chesta chaliye jao.
BANGLA:=>> <BOS>যদি টা

In [81]:
texts = [
    "O baba! Ei toh amar purono diary.",
    "Bus ta chole gelo, ami ekhono rastay dariye.",
    "Shondhya bela, meghla akash, aar ek cup cha.",
    "Tomar haashi ta aj onno rokom lagchhe.",
    "Ei golpota shotti na mittha, ke jane!",
    "Shey abar asbe na, bujhte parini ami.",
    "Chor palie gelo, kintu police ekdom ready chhilo.",
    "Ami jani na keno, kintu mone hoy bhul korechhi.",
    "Bristi porchhe, aar ami chatri niye beriyechi!",
    "Kichu kichu kotha thake mone, jodio somoy chole jay.",
    "Chhuti diner shokal, bed theke uthte iccha kore na.",
    "Laptop ta abar hang korlo, ekhono kaj sesh hoyni.",
    "Misti kheye khub moja holo, kintu choker jol thamlo na.",
    "Shey bollo, 'Ami chole jabo', ar ami kichu bolte parlam na.",
    "Ghore pashbalish nei, raat ta ki bhabe katbo!",
    "Tomar chokher ei bhasha ami bujhte shikhechi.",
    "Ei notun gaan ta khub touching!",
    "Train ta 10 minute deri te chilo, tai bus niyechilam.",
    "Pet kharap, doctor bollo kichu khawa jabe na.",
    "Aj tar birthday, kintu shey shobai theke dure.",
    "Chhoto belar golpo gulo mone pore, chokhe jol eshe jay.",
    "Tumi chilo amar shopner modhye ekta sundor golpo.",
    "Ghum bhenge gelo, kintu shopno ta mone nei.",
    "Kichu din ghure ashi? Mon ta khub bhore ache.",
    "Rickshaw wala bollo, 'Babu, ekhane jam achhe!'",
    "Cinema ta khub boring chhilo, ami majhe beriye elam.",
    "Chhele ta chocolate kheye hath bhenge felllo!",
    "Shey amar dike takiye bollo, 'Bhalo thako'.",
    "Aj school e teacher aseni, sobai khushi!",
    "Chand uthechhe, brishti porchhe, raat khub romantic.",
    "Taka ta kothay gelo? Ami to ektu aagei rekhechhilam!",
    "Ei notun mobile ta khub smart, kintu battery khub kom chole.",
    "Tumi je amar chhobi ta tulchho, eta boloni toh!",
    "Jam chhilo na, kintu ami deri korei beriyechilam.",
    "Kichu kichu gaan mone boro ekta gopon kotha mone koray.",
    "Shey chhilo amar pasher barir meye, ekhon tar kono khobor nei.",
    "Coffee ta thanda hoye gelo, kintu tumi toh asle na!",
    "Akasher oi megh gulo dekhle mone hoy ami urte pari.",
    "Kalke raat e brishti chhilo, tar sathe chhilo golpo.",
    "Nijer sathe nijer loraai chhilo sobcheye kothin.",
    "Bari phire dekhlam, sob kichu othe jemon chhilo.",
    "Ajke tomar mukh dekhe din ta valo lagchhe!",
    "Shohor ta porichito, kintu manush gulo noy.",
    "Tui abar hath ketechhis? Ar koto bar bolbo?",
    "Ekbar mathaye jodi kichu dhuke, ber hoy na!",
    "Tumi ki ekhono bhabcho amar kotha?",
    "Aj ami nijeke ekta chhoto boyosh er chhele mone korchi.",
    "Tui chhere gechhis, kintu mone toh achhis!",
    "Ei poth ta je kothay je niye jabe, ami jani na.",
    "Ami tomar naam ta ekbar shunei bhalobeshechhilam.",
    "Paharer opor theke surjo dube jawa dekhte bhalo lage."
]
for text in texts:
    print('===================================================================================================================================')
    print("BANGLISH:=>>", text)
    print("BANGLA:=>>",transformer.transliterate(text, max_output_len=256, stop_idx=3))
    print('===================================================================================================================================')

BANGLISH:=>> O baba! Ei toh amar purono diary.
BANGLA:=>> <BOS>ও বাবা! এই তোহ আমার পুর্ণ দিয়ার।<EOS>
BANGLISH:=>> Bus ta chole gelo, ami ekhono rastay dariye.
BANGLA:=>> <BOS>বাস টা চলে গেলো, আমি এখনো রাস্টায় দারিয়ে।<EOS>
BANGLISH:=>> Shondhya bela, meghla akash, aar ek cup cha.
BANGLA:=>> <BOS>সন্দেখা বেলা, মেগা একসা আশা এর এক পুর চাচ্ছা। <EOS>
BANGLISH:=>> Tomar haashi ta aj onno rokom lagchhe.
BANGLA:=>> <BOS>তোমার হাসি টা আজ অন্য রমোক লাগছে। <EOS>
BANGLISH:=>> Ei golpota shotti na mittha, ke jane!
BANGLA:=>> <BOS>এই গল্পটা সহত্তি না মিথ্যা, কে জানে!!<EOS>
BANGLISH:=>> Shey abar asbe na, bujhte parini ami.
BANGLA:=>> <BOS>সেই আবার আসবে না, বুঝতে পারিনি আমি। <EOS>
BANGLISH:=>> Chor palie gelo, kintu police ekdom ready chhilo.
BANGLA:=>> <BOS>চোর পালি গেলো, কিন্তু পলাইস একদম দের এ চালি ছিল।<EOS>
BANGLISH:=>> Ami jani na keno, kintu mone hoy bhul korechhi.
BANGLA:=>> <BOS>আমি জানি না কেন, কিন্তু মনে হয় ভাল করেছি। <EOS>
BANGLISH:=>> Bristi porchhe, aar ami chatri niye beriyechi!
BANGLA:

In [107]:
# Tried some bengali slangs. Kindly ignore
texts = [
    'amar pood e kathhi korle amio dekhiye debo',
    'bes Harami manush to!!'
]
for text in texts:
    print('===================================================================================================================================')
    print("BANGLISH:=>>", text)
    print("BANGLA:=>>",transformer.transliterate(text, max_output_len=256, stop_idx=3))
    print('===================================================================================================================================')

BANGLISH:=>> amar pood e kathhi korle amio dekhiye debo
BANGLA:=>> <BOS>আমার পদ এ কাটি করলে আমিও দেখিয়ে দেবো<EOS>
BANGLISH:=>> bes Harami manush to!!
BANGLA:=>> <BOS>বেশ হারামি মানুষি তো!!!!<EOS>
